# Derivatives Assignment

## Read Data

In [9]:
import pandas as pd
import numpy as np

df = pd.read_json('data/portfolio_dailies.json')
df.head()
#Index is Date


,JMCO,BAUSM,CISK,JMVI
2020-03-25,276.24,139.61,133.6187,160.83
2020-03-26,276.51,140.34,133.5489,161.44
2020-03-27,276.78,140.36,133.6695,160.39
2020-03-30,277.10,140.80,132.7614,163.36
2020-03-31,277.03,140.83,132.7550,164.46


#### Find the following on the individuals stocks:
#### Annual returns, volatility, upvol, down vol, sharpe, sortino, calmar, realized correlation matrix

In [10]:
# Calculate daily returns
returns = df.pct_change()

# Calculate annual returns
annual_returns = returns.mean() * 252

# Calculate volatility
volatility = returns.std() * np.sqrt(252)

# Calculate up and down vol
up_vol = returns[returns > 0].std() * np.sqrt(252)
down_vol = returns[returns < 0].std() * np.sqrt(252)

# Calculate Sharpe ratio
risk_free_rate = 0.02  # Assuming a risk-free rate of 2%
sharpe_ratio = (annual_returns - risk_free_rate) / volatility

# Calculate Sortino ratio
target_return = 0  # Assuming target return of 0
sortino_ratio = (annual_returns - target_return) / down_vol

# Calculate Calmar ratio
max_drawdown = np.abs(returns.min())
calmar_ratio = annual_returns / max_drawdown

# Calculate realized correlation matrix
correlation_matrix = returns.corr()

# Display results
print("Annual Returns:")
print(annual_returns)
print("\nVolatility:")
print(volatility)
print("\nUp Volatility:")
print(up_vol)
print("\nDown Volatility:")
print(down_vol)
print("\nSharpe Ratio:")
print(sharpe_ratio)
print("\nSortino Ratio:")
print(sortino_ratio)
print("\nCalmar Ratio:")
print(calmar_ratio)
print("\nRealized Correlation Matrix:")
print(correlation_matrix)

Annual Returns:
JMCO     0.062745
BAUSM    0.037906
CISK     0.073335
JMVI     0.140500
dtype: float64

Volatility:
JMCO     0.026561
BAUSM    0.020511
CISK     0.045748
JMVI     0.070840
dtype: float64

Up Volatility:
JMCO     0.017896
BAUSM    0.014281
CISK     0.040783
JMVI     0.038194
dtype: float64

Down Volatility:
JMCO     0.017338
BAUSM    0.021245
CISK     0.028427
JMVI     0.096738
dtype: float64

Sharpe Ratio:
JMCO     1.609293
BAUSM    0.872975
CISK     1.165842
JMVI     1.701005
dtype: float64

Sortino Ratio:
JMCO     3.618938
BAUSM    1.784200
CISK     2.579808
JMVI     1.452372
dtype: float64

Calmar Ratio:
JMCO     8.051013
BAUSM    2.221313
CISK     4.638904
JMVI     1.833562
dtype: float64

Realized Correlation Matrix:
           JMCO     BAUSM      CISK      JMVI
JMCO   1.000000  0.021729 -0.023430  0.017106
BAUSM  0.021729  1.000000 -0.160316  0.239222
CISK  -0.023430 -0.160316  1.000000 -0.168380
JMVI   0.017106  0.239222 -0.168380  1.000000


#### Equally weighted basket metrics

In [11]:
# Calculate equally weighted portfolio metrics
weights = np.array([0.25, 0.25, 0.25, 0.25])  # Equal weights for each stock
portfolio_returns = np.dot(annual_returns, weights)
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(correlation_matrix, weights))) * np.sqrt(252)
portfolio_sharpe_ratio = (portfolio_returns - risk_free_rate) / portfolio_volatility
portfolio_sortino_ratio = (portfolio_returns - target_return) / down_vol.mean()
portfolio_max_drawdown = np.abs(returns.sum(axis=1).min())
portfolio_calmar_ratio = portfolio_returns / portfolio_max_drawdown

# Print the results
print("\nEqually Weighted Portfolio Metrics:")
print("Portfolio Annual Return:", portfolio_returns)
print("Portfolio Volatility:", portfolio_volatility)
print("Portfolio Sharpe Ratio:", portfolio_sharpe_ratio)
print("Portfolio Sortino Ratio:", portfolio_sortino_ratio)
print("Portfolio Calmar Ratio:", portfolio_calmar_ratio)



Equally Weighted Portfolio Metrics:
Portfolio Annual Return: 0.07862134688000716
Portfolio Volatility: 7.7888894800455315
Portfolio Sharpe Ratio: 0.007526277915509012
Portfolio Sortino Ratio: 1.9205467702212533
Portfolio Calmar Ratio: 1.0813487467096028


#### 25M Notional , 10% Return Volatility Calculations

#### Mean-Variance Optimization 

In [12]:
from scipy.optimize import minimize

# Function to minimize for MVO & target vol of 10%
def objective(weights):
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(correlation_matrix, weights))) * np.sqrt(252)
    return (portfolio_volatility - 0.1) ** 2 

# Constraints for weights summing to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Initial guess for weights
initial_guess = np.ones(len(weights)) / len(weights)

# Perform optimization & convert weights to notional
result = minimize(objective, initial_guess, constraints=constraints)
optimized_weights_mvo = result.x * 25e6  

print("\nMean-Variance Optimization (MVO) Weights (Notional):")
print("JMCO:", optimized_weights_mvo[0])
print("BAUSM:", optimized_weights_mvo[1])
print("CISK:", optimized_weights_mvo[2])
print("JMVI:", optimized_weights_mvo[3])



Mean-Variance Optimization (MVO) Weights (Notional):
JMCO: 5812546.116382226
BAUSM: 5610115.357692426
CISK: 7848715.285026995
JMVI: 5728623.240898351


#### Risk Parity & Minimum Variance Optimization

In [13]:
# Risk Parity
def risk_parity(weights):
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(correlation_matrix, weights))) * np.sqrt(252)
    risk_contributions = np.dot(correlation_matrix, weights) * weights
    risk_budget = 1 / len(weights)
    return np.sum((risk_contributions - risk_budget) ** 2)  # Minimize deviation from risk budget

result_rp = minimize(risk_parity, initial_guess, constraints=constraints)
optimized_weights_rp = result_rp.x * 25e6  # Convert weights to notional

print("\nRisk Parity Weights (Notional):")
print("JMCO:", optimized_weights_rp[0])
print("BAUSM:", optimized_weights_rp[1])
print("CISK:", optimized_weights_rp[2])
print("JMVI:", optimized_weights_rp[3])

# Minimum Variance Portfolio
def min_variance(weights):
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(correlation_matrix, weights))) * np.sqrt(252)
    return portfolio_volatility

result_mv = minimize(min_variance, initial_guess, constraints=constraints)
optimized_weights_mv = result_mv.x * 25e6  # Convert weights to notional

print("\nMinimum Variance Portfolio Weights (Notional):")
print("JMCO:", optimized_weights_mv[0])
print("BAUSM:", optimized_weights_mv[1])
print("CISK:", optimized_weights_mv[2])
print("JMVI:", optimized_weights_mv[3])



Risk Parity Weights (Notional):
JMCO: 12794741.689064609
BAUSM: 11041852.422683656
CISK: -9861998.647121316
JMVI: 11025404.53537305

Minimum Variance Portfolio Weights (Notional):
JMCO: 5812544.04422289
BAUSM: 5610115.6764426
CISK: 7848715.222873613
JMVI: 5728625.056460895


#### Black Scholes Option Pricer

In [14]:
import math
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    call_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    return call_price

def black_scholes_put(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (in years)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility

call_price = black_scholes_call(S, K, T, r, sigma)
put_price = black_scholes_put(S, K, T, r, sigma)

print("Black-Scholes Call Option Price:", call_price)
print("Black-Scholes Put Option Price:", put_price)


Black-Scholes Call Option Price: 10.450583572185565
Black-Scholes Put Option Price: 5.573526022256971


#### Black Scholes - Monte Carlo Model

In [15]:
# BSM Monte-Carlo
import scipy.stats as stats

def black_scholes_mc(S, K, T, r, sigma, num_simulations=10000):
    np.random.seed(42)
    z = np.random.normal(0, 1, num_simulations)
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    
    call_payoffs = np.maximum(ST - K, 0)
    put_payoffs = np.maximum(K - ST, 0)
    
    call_price = np.exp(-r * T) * np.mean(call_payoffs)
    put_price = np.exp(-r * T) * np.mean(put_payoffs)
    
    return call_price, put_price

S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (in years)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility

call_price_mc, put_price_mc = black_scholes_mc(S, K, T, r, sigma)

print("Monte Carlo Call Option Price:", call_price_mc)
print("Monte Carlo Put Option Price:", put_price_mc)


Monte Carlo Call Option Price: 10.450169921134655
Monte Carlo Put Option Price: 5.601717175958082


#### European Digital Options Pricer

In [16]:
def norm_cdf(x):
    return (1 + math.erf(x / math.sqrt(2))) / 2

def european_digital_call(S, K, T, r, sigma, payout):
    d2 = (math.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    digital_price = payout * norm_cdf(d2)
    return digital_price * math.exp(-r * T)

def european_digital_put(S, K, T, r, sigma, payout):
    d2 = (math.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    digital_price = payout * norm_cdf(-d2)
    return digital_price * math.exp(-r * T)

S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (in years)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
payout = 100  # Payout amount if option expires in-the-money

digital_call_price = european_digital_call(S, K, T, r, sigma, payout)
digital_put_price = european_digital_put(S, K, T, r, sigma, payout)

print("European Digital Call Option Price:", digital_call_price)
print("European Digital Put Option Price:", digital_put_price)


European Digital Call Option Price: 53.23248154537634
European Digital Put Option Price: 41.89046090469506
